# Taxi CrossValidation with GPU accelerating on XGBoost

In this notebook, we will show you how to levarage GPU to accelerate taxi CrossValidation on XGBoost to find out the best model given a group parameters.

## Import classes
First we need load some common classes that both GPU version and CPU version will use:

In [1]:
import ml.dmlc.xgboost4j.scala.spark.{XGBoostRegressionModel, XGBoostRegressor}
import org.apache.spark.ml.evaluation.{RegressionEvaluator}
import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types.{FloatType, IntegerType, StructField, StructType}

what is new to xgboost-spark users is rapids.GpuDataReader and **rapids.CrossValidator**

In [2]:
import ml.dmlc.xgboost4j.scala.spark.rapids.{CrossValidator, GpuDataReader}

## Set dataset path

In [3]:
val trainParquetPath="/data/taxi/parquet/train"
val evalParquetPath="/data/taxi/parquet/eval"

trainParquetPath = /data/taxi/parquet/train
evalParquetPath = /data/taxi/parquet/eval


/data/taxi/parquet/eval

# Set the schema of the dataset

In [4]:
val labelColName = "fare_amount"
val schema =
    StructType(Array(
      StructField("vendor_id", FloatType),
      StructField("passenger_count", FloatType),
      StructField("trip_distance", FloatType),
      StructField("pickup_longitude", FloatType),
      StructField("pickup_latitude", FloatType),
      StructField("rate_code", FloatType),
      StructField("store_and_fwd", FloatType),
      StructField("dropoff_longitude", FloatType),
      StructField("dropoff_latitude", FloatType),
      StructField(labelColName, FloatType),
      StructField("hour", FloatType),
      StructField("year", IntegerType),
      StructField("month", IntegerType),
      StructField("day", FloatType),
      StructField("day_of_week", FloatType),
      StructField("is_weekend", FloatType)
    ))

labelColName = fare_amount
schema = StructType(StructField(vendor_id,FloatType,true), StructField(passenger_count,FloatType,true), StructField(trip_distance,FloatType,true), StructField(pickup_longitude,FloatType,true), StructField(pickup_latitude,FloatType,true), StructField(rate_code,FloatType,true), StructField(store_and_fwd,FloatType,true), StructField(dropoff_longitude,FloatType,true), StructField(dropoff_latitude,FloatType,true), StructField(fare_amount,FloatType,true), StructField(hour,FloatType,true), StructField(year,IntegerType,true), StructField(month,IntegerType,true), StructField(day,FloatType,true), StructField(day_of_week,FloatType,true), StructField(is_weekend,FloatType,true))


StructType(StructField(vendor_id,FloatType,true), StructField(passenger_count,FloatType,true), StructField(trip_distance,FloatType,true), StructField(pickup_longitude,FloatType,true), StructField(pickup_latitude,FloatType,true), StructField(rate_code,FloatType,true), StructField(store_and_fwd,FloatType,true), StructField(dropoff_longitude,FloatType,true), StructField(dropoff_latitude,FloatType,true), StructField(fare_amount,FloatType,true), StructField(hour,FloatType,true), StructField(year,IntegerType,true), StructField(month,IntegerType,true), StructField(day,FloatType,true), StructField(day_of_week,FloatType,true), StructField(is_weekend,FloatType,true))

## Create a new spark session and load data
we must create a new spark session to continue all spark operations. It will also be used to initilize the `GpuDataReader` which is a data reader powered by GPU.

NOTE: in this notebook, we have uploaded dependency jars when installing toree kernel. If we don't upload them at installation time, we can also upload in notebook by [%AddJar magic](https://toree.incubator.apache.org/docs/current/user/faq/). However, there's one restriction for `%AddJar`: the jar uploaded can only be available when `AddJar` is called after a new spark session is created. We must use it as below:

```scala
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder().appName("Taxi-GPU").getOrCreate
%AddJar file:/data/libs/cudf-0.9.2-cuda10.jar
%AddJar file:/data/libs/xgboost4j_2.x-1.0.0-Beta5.jar
%AddJar file:/data/libs/xgboost4j-spark_2.x-1.0.0-Beta5.jar
// ...
```

In [5]:
val spark = SparkSession.builder().appName("taxi-gpu-cv").getOrCreate()
val dataReader = new GpuDataReader(spark)
val trainDs = dataReader.parquet(trainParquetPath)

spark = org.apache.spark.sql.SparkSession@403e9f64
dataReader = ml.dmlc.xgboost4j.scala.spark.rapids.GpuDataReader@76d0ba31
trainDs = ml.dmlc.xgboost4j.scala.spark.rapids.GpuDataset@459d462


ml.dmlc.xgboost4j.scala.spark.rapids.GpuDataset@459d462

## Find out features to train

In [6]:
val featureNames = schema.filter(_.name != labelColName).map(_.name)

featureNames = List(vendor_id, passenger_count, trip_distance, pickup_longitude, pickup_latitude, rate_code, store_and_fwd, dropoff_longitude, dropoff_latitude, hour, year, month, day, day_of_week, is_weekend)


List(vendor_id, passenger_count, trip_distance, pickup_longitude, pickup_latitude, rate_code, store_and_fwd, dropoff_longitude, dropoff_latitude, hour, year, month, day, day_of_week, is_weekend)

In [7]:
val regressorParam = Map(
    "learning_rate" -> 0.05,
    "max_depth" -> 8,
    "subsample" -> 0.8,
    "gamma" -> 1,
    "num_round" -> 100,
    "tree_method" -> "gpu_hist")

regressorParam = Map(learning_rate -> 0.05, subsample -> 0.8, max_depth -> 8, num_round -> 100, tree_method -> gpu_hist, gamma -> 1)


Map(learning_rate -> 0.05, subsample -> 0.8, max_depth -> 8, num_round -> 100, tree_method -> gpu_hist, gamma -> 1)

## Construct CrossValidator

In [8]:
val regressor = new XGBoostRegressor(regressorParam)
    .setLabelCol(labelColName)
    .setFeaturesCols(featureNames)
val paramGrid = new ParamGridBuilder()
    .addGrid(regressor.maxDepth, Array(3, 10))
    .addGrid(regressor.eta, Array(0.2, 0.6))
    .build()
val evaluator = new RegressionEvaluator().setLabelCol(labelColName)
val cv = new CrossValidator()
    .setEstimator(regressor)
    .setEvaluator(evaluator)
    .setEstimatorParamMaps(paramGrid)
    .setNumFolds(3)

regressor = xgbr_95e3889fb8ff
paramGrid = 
evaluator = regEval_3a1d557767ab
cv = cv_7d300972c3cb


Array({
	xgbr_95e3889fb8ff-eta: 0.2,
	xgbr_95e3889fb8ff-maxDepth: 3
}, {
	xgbr_95e3889fb8ff-eta: 0.2,
	xgbr_95e3889fb8ff-maxDepth: 10
}, {
	xgbr_95e3889fb8ff-eta: 0.6,
	xgbr_95e3889fb8ff-maxDepth: 3
}, {
	xgbr_95e3889fb8ff-eta: 0.6,
	xgbr_95e3889fb8ff-maxDepth: 10
})


cv_7d300972c3cb

## train with CrossValidator

In [9]:
val model = cv.fit(trainDs).asInstanceOf[XGBoostRegressionModel]

Tracker started, with env={DMLC_NUM_SERVER=0, DMLC_TRACKER_URI=10.19.183.93, DMLC_TRACKER_PORT=9094, DMLC_NUM_WORKER=1}
Tracker started, with env={DMLC_NUM_SERVER=0, DMLC_TRACKER_URI=10.19.183.93, DMLC_TRACKER_PORT=9094, DMLC_NUM_WORKER=1}
Tracker started, with env={DMLC_NUM_SERVER=0, DMLC_TRACKER_URI=10.19.183.93, DMLC_TRACKER_PORT=9094, DMLC_NUM_WORKER=1}
Tracker started, with env={DMLC_NUM_SERVER=0, DMLC_TRACKER_URI=10.19.183.93, DMLC_TRACKER_PORT=9094, DMLC_NUM_WORKER=1}
Tracker started, with env={DMLC_NUM_SERVER=0, DMLC_TRACKER_URI=10.19.183.93, DMLC_TRACKER_PORT=9094, DMLC_NUM_WORKER=1}
Tracker started, with env={DMLC_NUM_SERVER=0, DMLC_TRACKER_URI=10.19.183.93, DMLC_TRACKER_PORT=9094, DMLC_NUM_WORKER=1}
Tracker started, with env={DMLC_NUM_SERVER=0, DMLC_TRACKER_URI=10.19.183.93, DMLC_TRACKER_PORT=9094, DMLC_NUM_WORKER=1}
Tracker started, with env={DMLC_NUM_SERVER=0, DMLC_TRACKER_URI=10.19.183.93, DMLC_TRACKER_PORT=9094, DMLC_NUM_WORKER=1}
Tracker started, with env={DMLC_NUM_SERV

model = xgbr_95e3889fb8ff


xgbr_95e3889fb8ff

## tranform with best model trained by CrossValidator

In [10]:
val transformDs = dataReader.parquet(evalParquetPath)
val df = model.transform(transformDs).cache()
df.select("fare_amount", "prediction").show(5)

+-----------+------------------+
|fare_amount|        prediction|
+-----------+------------------+
|        8.9| 9.272931098937988|
|       23.7| 24.58644676208496|
|       14.9|12.472931861877441|
|        3.7| 4.018008232116699|
|        6.1| 6.843854904174805|
+-----------+------------------+
only showing top 5 rows



transformDs = ml.dmlc.xgboost4j.scala.spark.rapids.GpuDataset@3e18dab1
df = [vendor_id: float, passenger_count: float ... 17 more fields]


[vendor_id: float, passenger_count: float ... 17 more fields]

In [11]:
val evaluator = new RegressionEvaluator().setLabelCol(labelColName)
val rmse = evaluator.evaluate(df)
println("RMSE -- " + rmse)

RMSE -- 3.3533535694226924


evaluator = regEval_d7dabcdd2440
rmse = 3.3533535694226924


3.3533535694226924

In [12]:
spark.close()